In [121]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from keras.optimizers import Adam
import cv2 as cv
from PIL import Image

In [122]:
# Load the data
split_df = pd.read_csv('split_df.csv', encoding='utf-8-sig')


In [123]:
split_df

,Table 1,Spectacles,Age,Gender,Name,id,Eye,VA,CDR,Diagnosis,IOP,Image
0,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,زرق أيمن تم معالجته جراحيا,0.0,64,0,شاهين ميرو,1.0,1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
2,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,0,NaN,NaN,1,25.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
3,زرق متقدم,1.0,62,0,أيمن سمارة,2.0,1,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
4,لا يوجد زرق,0.0,48,1,رندة ناصر,3.0,0,1.0,0.5,0,15.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
85,زرق أيسر تم معالجته جراحيا,1.0,55,1,زريفة المحمد,43.0,1,NaN,NaN,0,NaN,Data/BASEL_BASH Optic nerve.png_ left.png
86,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,0,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png
87,مريض زرقي ثانوي بعد استخراج ساد لا يوجد اصابة ...,0.0,71,0,هيثم قزاز,44.0,1,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png
88,لا يوجد زرق,1.0,65,0,حمود غشام,45.0,0,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png


In [124]:
split_df = split_df.drop(['Table 1', 'Gender', 'id', 'Age', 'Spectacles'], axis=1)

In [125]:
split_df = split_df.drop(['Name','Eye'], axis=1)

In [126]:
split_df.dropna(inplace=True)

In [127]:
split_df

,VA,CDR,Diagnosis,IOP,Image
0,0.6,0.5,0,14.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
1,0.7,0.9,1,23.0,Data/SHAHEN_MERO Optic nerve.png_ left.png
3,9.0,0.8,1,40.0,Data/SHAHEN_MERO Optic nerve.png_ right.png
6,0.2,0.7,1,10.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
7,0.1,0.7,1,14.0,Data/AYMAN_SAMARA_20240121_124719_Disc_3D_R_SI...
...,...,...,...,...,...
84,0.2,0.8,1,10.0,Data/BASEL_BASH Optic nerve.png_ left.png
86,0.3,0.5,1,15.0,Data/BASEL_BASH Optic nerve.png_ right.png
87,0.6,0.7,1,16.0,Data/BASEL_BASH Optic nerve.png_ right.png
88,0.5,0.3,0,15.0,Data/NEZAR_HELOU Optic nerve.png_ left.png


In [128]:
len(split_df['Image'].index)

66

In [129]:
split_df['Image'][0]

'Data/SHAHEN_MERO Optic nerve.png_ left.png'

In [130]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Preprocess image function
# def preprocess_image(image_path, target_size=(224, 224)):
#     # Read the image
#     image = cv2.imread(image_path)
    
#     # Resize the image
#     image = cv2.resize(image, target_size)
    
#     # Convert the image to float32
#     image = image.astype(np.float32)
    
#     # Normalize pixel values to [0, 1]
#     image /= 255.0
    
#     return image

# Concatenation model
def create_concat_model(image_shape, num_numerical_features, num_classes):
    # Define input layers for images and numerical features
    image_input = Input(shape=image_shape)
    numerical_input = Input(shape=(num_numerical_features,))
    
    # Image processing layers
    conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu')(pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    flatten_img = Flatten()(pool2)
    
    # Dense layers for numerical features
    dense1 = Dense(64, activation='relu')(numerical_input)
    
    # Concatenate image and feature layers
    concatenated = Concatenate()([flatten_img, dense1])
    
    # Additional layers for further processing or output
    dense2 = Dense(128, activation='relu')(concatenated)
    output = Dense(num_classes, activation='sigmoid')(dense2)
    
    # Define the model
    model = tf.keras.Model(inputs=[image_input, numerical_input], outputs=output)
    
    return model


# Data augmentation
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


augmented_images = []

for index,row in split_df.iterrows():
    # Load an image and preprocess it
    path = row['Image']
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (224, 224))  # Resize the image to the desired size
    image = image.astype(np.float32) / 255.0  # Normalize pixel values to [0, 1]
    augmented_images.append(image)


    # # Apply data augmentation
    # for _ in range(5):  # Generate 5 augmented images
    #     augmented_image = data_generator.random_transform(image)
    #     augmented_images.append(augmented_image)





In [131]:
numerical_features = split_df.drop(['Diagnosis','Image'],axis=1)
numerical_features = np.array(numerical_features)
labels = np.array(split_df['Diagnosis'])  # NumPy array of shape (num_samples,)


In [132]:
numerical_features.shape

(66, 3)

In [133]:
len(numerical_features)

66

In [134]:
from keras.backend import clear_session
clear_session()

In [135]:
# Example usage:
image_shape = augmented_images[0].shape 
num_numerical_features = 3 
num_classes = 1  
concat_model = create_concat_model(image_shape, num_numerical_features, num_classes)

# Compile the model
concat_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [136]:
concat_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 222, 222, 32)         896       ['input_1[0][0]']             
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)         0         ['conv2d[0][0]']              
 D)                                                                                               
                                                                                                  
 conv2d_1 (Conv2D)           (None, 109, 109, 64)         18496     ['max_pooling2d[0][0]']   

In [137]:
# Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [138]:
X_image_train, X_image_test, X_num_features_train, X_num_features_test, y_labels_train, y_labels_test = train_test_split(augmented_images, numerical_features, labels, test_size=0.1, random_state=42)

In [139]:
print(len(X_image_train))
print(len(X_image_test))
print(len(X_num_features_train))
print(len(X_num_features_test))
print(len(y_labels_train))
print(len(y_labels_test))

59
7
59
7
59
7


In [140]:
X_image_train, X_image_test, X_num_features_train, X_num_features_test, y_labels_train, y_labels_test = \
    train_test_split(augmented_images, numerical_features, labels, test_size=0.2, random_state=42)


print(len(X_image_train))
print(len(X_image_test))
print(len(X_num_features_train))
print(len(X_num_features_test))
print(len(y_labels_train))
print(len(y_labels_test))


52
14
52
14
52
14


In [141]:
y_labels_train.size

52

In [142]:
x=[X_image_train, X_num_features_train]

In [143]:
type(x)

list

In [144]:
import numpy as np  # تأكد من استيراد مكتبة numpy

# تحويل القوائم إلى مصفوفات numpy
X_image_train_np = np.array(X_image_train)
X_num_features_train_np = np.array(X_num_features_train)
y_labels_train_np = np.array(y_labels_train)

# طباعة الأبعاد
print(X_image_train_np.shape, X_num_features_train_np.shape, y_labels_train_np.shape)


(52, 224, 224, 3) (52, 3) (52,)


In [145]:
concat_model.fit(x=[np.array(X_image_train), X_num_features_train], y=y_labels_train, validation_data=([np.array(X_image_test), X_num_features_test], y_labels_test),
                  batch_size=32, epochs=10, )

Epoch 1/10


2/2 [==============================] - 3s 1s/step - loss: 4.9263 - accuracy: 0.5769 - val_loss: 1.2020 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 2s 855ms/step - loss: 1.6404 - accuracy: 0.7308 - val_loss: 3.6957 - val_accuracy: 0.3571
Epoch 3/10
2/2 [==============================] - 2s 856ms/step - loss: 1.9167 - accuracy: 0.5192 - val_loss: 0.7059 - val_accuracy: 0.6429
Epoch 4/10
2/2 [==============================] - 2s 874ms/step - loss: 0.9048 - accuracy: 0.5769 - val_loss: 0.8147 - val_accuracy: 0.6429
Epoch 5/10
2/2 [==============================] - 2s 942ms/step - loss: 0.8410 - accuracy: 0.5769 - val_loss: 0.6055 - val_accuracy: 0.5714
Epoch 6/10
2/2 [==============================] - 2s 844ms/step - loss: 0.4172 - accuracy: 0.8269 - val_loss: 1.1145 - val_accuracy: 0.5000
Epoch 7/10
2/2 [==============================] - 2s 848ms/step - loss: 0.4184 - accuracy: 0.8077 - val_loss: 1.0147 - val_accuracy: 0.5714
Epoch 8/10
2/2 [==================

In [146]:
concat_model.fit(x=[np.array(X_image_train), X_num_features_train], y=y_labels_train, validation_data=([np.array(X_image_test), X_num_features_test], y_labels_test),
                  batch_size=32, epochs=10, )

Epoch 1/10
2/2 [==============================] - 2s 834ms/step - loss: 0.2082 - accuracy: 0.8846 - val_loss: 1.2372 - val_accuracy: 0.5714
Epoch 2/10
2/2 [==============================] - 2s 800ms/step - loss: 0.1665 - accuracy: 0.9038 - val_loss: 1.1916 - val_accuracy: 0.5714
Epoch 3/10
2/2 [==============================] - 2s 813ms/step - loss: 0.1573 - accuracy: 0.8846 - val_loss: 1.4698 - val_accuracy: 0.5714
Epoch 4/10
2/2 [==============================] - 2s 832ms/step - loss: 0.1683 - accuracy: 0.9615 - val_loss: 1.7097 - val_accuracy: 0.5714
Epoch 5/10
2/2 [==============================] - 2s 838ms/step - loss: 0.1258 - accuracy: 0.9615 - val_loss: 1.6987 - val_accuracy: 0.5714
Epoch 6/10
2/2 [==============================] - 2s 749ms/step - loss: 0.1627 - accuracy: 0.9423 - val_loss: 1.8010 - val_accuracy: 0.5714
Epoch 7/10
2/2 [==============================] - 2s 735ms/step - loss: 0.1570 - accuracy: 0.9038 - val_loss: 2.0660 - val_accuracy: 0.5714
Epoch 8/10
2/2 [====